# ProPyCore SDK Examples: Budgets

This notebook contains snippets from the `budgets.py` module.

In [1]:
import os
import dotenv
import json

import ProPyCore

## Setup
You will need to create the connection to your Procore app and then get the details for your company and project.

### Connection to Procore App
Ensure you have a `.env` file with the following information included:
* `CLIENT_ID`: your data connection app's client ID
* `CLIENT_SECRET`: your data connection app's client secret

In [2]:
dotenv.load_dotenv()
connection = ProPyCore.procore.Procore(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    redirect_uri="urn:ietf:wg:oauth:2.0:oob", # default for data connection apps
    oauth_url="https://app.procore.com", # default for data connection apps
    base_url="https://app.procore.com" # default for data connection apps
)

### Get Company and Project
Use the cells below to specify your company and your project

In [3]:
company_name = "Rogers-O`Brien Construction"
project_name = "Sandbox Test Project"

Now use the ProPyCore `connection` variable to get the company and project details.

In [4]:
company = connection.companies.find(identifier=company_name)
project = connection.projects.find(
    company_id=company["id"],
    identifier=project_name
)
print(company["id"], project["id"])

8089 2783683


---

## Budgets: Views
To get budget data, you need to specify which view you would like to access data from. On the frontend, the names of the views are provided in one of the dropdowns on the budget tool:

![budget-views](https://github.com/rogers-obrien-rad/ProPyCore/docs/images/procore-budget-views.png)

#### `budgets.views.get()`
You can get the same list of views as seen above using the `get()` method from the `budgets.views` accessor.

In [5]:
views = connection.budgets.views.get(
    company_id=company["id"],
    project_id=project["id"]
)
print(json.dumps(views[0], indent=4))

view_ids = [(view["name"], view["id"]) for view in views]
print("Budget view names and IDs:")
for view_id in view_ids:
    print(view_id)

{
    "id": 406809,
    "created_at": "2022-02-10T20:30:57Z",
    "description": "Detailed Budget View to be used for Forecasting",
    "name": "Detailed Budget View",
    "updated_at": "2022-03-10T21:37:23Z",
    "role": "budgeting",
    "links": {
        "detail_rows": "https://app.procore.com/rest/v1.0/budget_views/406809/detail_rows?project_id=2783683",
        "summary_rows": "https://app.procore.com/rest/v1.0/budget_views/406809/summary_rows?project_id=2783683"
    },
    "created_by": {
        "id": 6923972,
        "login": "david@mcdonnelconsulting.com",
        "name": "David McDonnel"
    }
}
Budget view names and IDs:
('Detailed Budget View', 406809)
('Non-Budgeted Sources', 408958)
('Change Management', 408960)
('Simple Budget View', 414205)
('RO Forecast Over Time', 428532)
('PSS Monthly Snapshot View', 445339)


#### `budgets.views.find()`
If you want to find a specific view, you can use the `find()` method.

In [6]:
# Use find() with a string name
view_str = "Detailed Budget View"
view_by_str = connection.budgets.views.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier=view_str
)
print(json.dumps(view_by_str, indent=4))

{
    "id": 406809,
    "created_at": "2022-02-10T20:30:57Z",
    "description": "Detailed Budget View to be used for Forecasting",
    "name": "Detailed Budget View",
    "updated_at": "2022-03-10T21:37:23Z",
    "role": "budgeting",
    "links": {
        "detail_rows": "https://app.procore.com/rest/v1.0/budget_views/406809/detail_rows?project_id=2783683",
        "summary_rows": "https://app.procore.com/rest/v1.0/budget_views/406809/summary_rows?project_id=2783683"
    },
    "created_by": {
        "id": 6923972,
        "login": "david@mcdonnelconsulting.com",
        "name": "David McDonnel"
    }
}


In [7]:
# Use find() with an integer ID
view_int= 414205
view_by_int = connection.budgets.views.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier=view_int
)
print(json.dumps(view_by_int, indent=4))

{
    "id": 414205,
    "created_at": "2022-03-10T20:44:29Z",
    "description": "Simplified budget view for PM's",
    "name": "Simple Budget View",
    "updated_at": "2022-03-10T21:37:41Z",
    "role": "budgeting",
    "links": {
        "detail_rows": "https://app.procore.com/rest/v1.0/budget_views/414205/detail_rows?project_id=2783683",
        "summary_rows": "https://app.procore.com/rest/v1.0/budget_views/414205/summary_rows?project_id=2783683"
    },
    "created_by": {
        "id": 6923972,
        "login": "david@mcdonnelconsulting.com",
        "name": "David McDonnel"
    }
}


If you provide an invalid string name or integer ID, the function will raise a `NotFoundItemError`.

In [8]:
_ = connection.budgets.views.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier=111111
)

NotFoundItemError: 'Could not find view 111111'

---

## Budgets: Columns
The endpoint to get column data from a budget view provides high-level meta data rather than actual numeric values. The columns that you can access relate to those on the Procore frontend:

![budget-cols](https://github.com/rogers-obrien-rad/ProPyCore/docs/images/procore-budget-cols.png)

#### `budgets.columns.get()`
You can get the list of columns for a budget view by using `get()` method from the `budgets.columns` accessor including the view ID.

:exclamation: **Note**: Only "source" and "standard" columns are accesible via the columns endpoint; "calculated" columns are not accessible.

In [ ]:
view_id = 414205 # simplified budget view from earlier example
columns = connection.budgets.columns.get(
    company_id=company["id"],
    project_id=project["id"],
    budget_view_id=view_id
)
col_ids = [(col["name"], col["id"]) for col in columns]
print(json.dumps(columns, indent=4))

print("Column names and IDs:")
for col_id in col_ids:
    print(col_id)

[
    {
        "id": "detail_type",
        "name": "Detail Type",
        "groupable": true,
        "aggregatable": false,
        "type": "standard",
        "filterable": true,
        "position": -1
    },
    {
        "id": "3141780",
        "name": "Commtiment Subcontracts",
        "aggregatable": true,
        "groupable": false,
        "position": 4,
        "type": "source",
        "filterable": false
    },
    {
        "id": "3141801",
        "name": "Commitment Purchase Orders",
        "aggregatable": true,
        "groupable": false,
        "position": 5,
        "type": "source",
        "filterable": false
    },
    {
        "id": "3141802",
        "name": "Commitment COs",
        "aggregatable": true,
        "groupable": false,
        "position": 6,
        "type": "source",
        "filterable": false
    },
    {
        "id": "3141791",
        "name": "Cost to Date",
        "aggregatable": true,
        "groupable": false,
        "position": 1,
  

#### `budgets.columns.find()`
If you know the `name` or `id` of the column, you can use the `find()` method from the `budgets.columns` accessor to get that column's specific details.

In [ ]:
# Use find() with a string name
col_str = "Cost to Date"
col_by_str = connection.budgets.columns.find(
    company_id=company["id"],
    project_id=project["id"],
    budget_view_id=view_id,
    identifier=col_str
)
print(json.dumps(col_by_str, indent=4))

{
    "id": "3141791",
    "name": "Cost to Date",
    "aggregatable": true,
    "groupable": false,
    "position": 1,
    "type": "source",
    "filterable": false
}


---

## Budgets: Rows

![budget-rows](https://github.com/rogers-obrien-rad/ProPyCore/docs/images/procore-budget-rows.png)

The rows contain the real budget data. 

#### `budgets.rows.get()`
You can get the list data by row for a budget view by using `get()` method from the `budgets.columns` accessor including the view ID.

In [9]:
view_id = 414205 # simplified budget view from earlier example
rows = connection.budgets.rows.get(
    company_id=company["id"],
    project_id=project["id"],
    budget_view_id=view_id
)
print(json.dumps(rows, indent=4))

[
    {
        "curve": null,
        "start_date": null,
        "end_date": null,
        "deletable": false,
        "budgeted": true,
        "budgetable": true,
        "budget_forecast": {
            "id": 34752781,
            "manual_amount": "0.00",
            "automatic_amount": "30000.00",
            "amount": "30000.00",
            "automatically_calculated": true,
            "calculation_strategy": "automatic",
            "notes": ""
        },
        "wbs_code": {
            "id": 1752071628,
            "description": "Project Manager.Labor",
            "flat_code": "00-.30.14-.L",
            "flat_name": "Project Manager.Labor",
            "segment_items": [
                {
                    "id": 971232400,
                    "code": "00-.30.14-",
                    "name": "Project Manager",
                    "path_code": "00-.30.14-",
                    "path_ids": [
                        971232362,
                        971232400
           